In [11]:
from mlat.dim2 import HexagonalLattice
hl = HexagonalLattice(k=[1., 1.], m=[1., 1., 1., 1., 1., 1.], precision=.1)
ws, evecs = hl.dispersion()
print(ws.shape)
print(evecs.shape)

G:\My Drive\Documents\School\卒論\code\mlat\dim2\hexagonal.py:91: RuntimeWarning:

invalid value encountered in sqrt



(63, 63, 12)
(63, 63, 12, 12)


# Dispersion relation

In [12]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Surface(z=ws[:, :, i], x=hl.qxs, y=hl.qys) for i in [7, 8]])
fig.show()

# Animation

In [13]:
import numpy as np
import plotly.graph_objects as go


# Parameters
qx = 2.658407
qy = 2.658407
Nx = 24
Ny = 12

idx = min(range(len(hl.qxs)), key=lambda i: abs(hl.qxs[i] - qx))
idy = min(range(len(hl.qys)), key=lambda i: abs(hl.qys[i] - qy))

Q = hl.Q(qx, qy)

mode = 6 # 0 to 8
w = ws[idy, idx, mode] # /s

grid = []
for y in range(Ny):
    x_pos = 0
    y_pos = np.sqrt(3) * (y + 1 / 2)
    inc_set = [[1 / 2, -np.sqrt(3) / 2], [1., 0.], [1 / 2, np.sqrt(3) / 2], [1., 0.]]
    row = []
    for x in range(Nx):
        row.append([x_pos, y_pos])
        x_pos += inc_set[x % 4][0]
        y_pos += inc_set[x % 4][1]
    grid.append(row)
grid = np.array(grid)

x = grid[:, :, 0].flatten()
y = grid[:, :, 1].flatten()
fig = go.Figure(
    data=[go.Scatter(x=x.real, y=y.real, mode='markers')],
    layout=go.Layout(
        title="Dispersion relation animation",
        yaxis=dict(scaleanchor="x", scaleratio=1),
    ),)
fig.show()

In [14]:
# Print infos
print("Angular frequency: ", w)
print("Wave number: \nqx: ", qx, "\nqy: ", qy)


def generate_dgrid(shape, A, B, C, D, E, F):
    dgrid = np.zeros(shape, dtype=np.complex128)

    # Process first 3 x 4 cell
    # Q2*C Q2*D Q1A Q1B
    # Q2*E B    C   Q1F
    # A    F    E   D
    dgrid[:3, :4] = np.array([
        [A, F, E, D],
        [Q[1].conj() * E, B, C, Q[0] * F],
        [Q[1].conj() * C, Q[1].conj() * D, Q[0] * A, Q[0] * B]
    ])

    for x in range(4, Nx):
        dgrid[:3, x] = dgrid[0, x - 4] * Q[2]
    for y in range(3, Ny):
        for x in range(Nx):
            dgrid[y, x] = dgrid[y - 3, x] * Q[0] * Q[1].conj()
    return dgrid

evecs_A = evecs[idy, idx, mode, :2]
evecs_B = evecs[idy, idx, mode, 2:4]
evecs_C = evecs[idy, idx, mode, 4:6]
evecs_D = evecs[idy, idx, mode, 6:8]
evecs_E = evecs[idy, idx, mode, 8:10]
evecs_F = evecs[idy, idx, mode, 10:]
dgrid = generate_dgrid(grid.shape, evecs_A, evecs_B, evecs_C, evecs_D, evecs_E, evecs_F)

Angular frequency:  1.4508646892873238
Wave number: 
qx:  2.658407 
qy:  2.658407


In [15]:
# Parameters for animation
fps = 30 # /s
animation_length = 3 # s

# Construct frames
frames = []
for t in range(int(animation_length * fps)):
    dt = t / fps
    dphase = dt * w * 2 * np.pi
    current_grid = grid + dgrid * np.exp(1.j * dphase)
    x = current_grid[:, :, 0].flatten()
    y = current_grid[:, :, 1].flatten()
    frames.append(go.Frame(data=[go.Scatter(x=x.real, y=y.real, mode='markers')]))

# Figure components
start_button = dict(
        label="Play",
        method="animate",
        args=[
            None, 
            {
                "frame":{"duration": 1000 / fps, "redraw": False},
                "fromcurrent": True, 
                "transition": {"duration": 100}
            }])
pause_button = dict(
        label="Pause",
        method="animate",
        args=[
            [None], 
            {
                "frame": {"duration": 0, "redraw": False},
                "mode": "immediate",
                "transition": {"duration": 0}
            }])

# Plot
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title="Dispersion relation animation",
        yaxis=dict(scaleanchor="x", scaleratio=1),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[start_button, pause_button
            ])
        ]
    ),
    frames=frames[1:])
fig.show()